In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

import pickle

/home/user/miniconda3/envs/bgem3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Load & Split

In [ ]:
class Dataset:
    def __init__(self, train_file_name, test_file_name):
        self.train_file_name = train_file_name
        self.test_file_name = test_file_name
        self.train_emb_list, self.test_emb_list = None, None
        self.text_list, self.feature_list = None, None
        self.train, self.test = None, None

        print(f"[Init] Dataset initialized with train: {train_file_name}, test: {test_file_name}")

    def get_cls_embedding_batch(self, texts, max_length=256, batch_size=32):
        print(f"[Embedding] Start embedding {len(texts)} texts (batch size: {batch_size})")
        MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModel.from_pretrained(MODEL_NAME)
        model.eval()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        embeddings = []
        with torch.no_grad():
            for i in tqdm(range(0, len(texts), batch_size), desc="[Embedding batches]"):
                batch_texts = texts[i:i+batch_size]
                inputs = tokenizer(
                    batch_texts,
                    return_tensors='pt',
                    truncation=True,
                    max_length=max_length,
                    padding='max_length'
                )
                inputs = {k: v.to(device) for k, v in inputs.items()}
                outputs = model(**inputs)
                cls_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
                embeddings.append(cls_emb)
        embeddings = np.vstack(embeddings) 
        print(f"[Embedding] Completed. Shape: {embeddings.shape}")
        return embeddings
    
    def load_train_csv(self):
        print(f"[Load] Loading train CSV: {self.train_file_name}")
        train = pd.read_csv(self.train_file_name)
        print(f"[Load] Loading test CSV: {self.test_file_name}")
        test = pd.read_csv(self.test_file_name)
        print(f"[Load] Loaded train shape: {train.shape}, test shape: {test.shape}")
        return train, test

    def load_train_pickle(self):
        print(f"[Load] Loading train pickle: {self.train_file_name}")
        with open(self.train_file_name, "rb") as f:
            train = pickle.load(f)
        print(f"[Load] Loading test pickle: {self.test_file_name}")
        with open(self.test_file_name, "rb") as f:
            test = pickle.load(f)
        self.train, self.test = train, test
        print(f"[Load] train_col_list: {list(train.columns)}")
        print(f"[Load] test_col_list: {list(test.columns)}")
        return train, test

    def set_list(self, train_emb_list, test_emb_list, feature_list, text_list=None):
        self.train_emb_list = train_emb_list
        self.test_emb_list = test_emb_list
        self.feature_list = feature_list
        self.text_list = text_list
        print(f"[Set] train_emb_list: {train_emb_list}, \n test_emb_list: {test_emb_list}, \nfeature_list: {feature_list}, \ntext_list: {text_list}")

    def split_data(self, train):
        print(f"[Split] Splitting data with stratify on 'generated'")
        col_list = self.train_emb_list + self.feature_list + self.text_list
        X = train[col_list]
        y = train['generated']
        X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
        print(f"[Split] X_train: {X_train.shape}, X_val: {X_val.shape}")
        return X_train, X_val, y_train, y_val
    
    def get_text_emb(self, text_list):
        if text_list is None:
            text_list = self.text_list
        text_set = []
        print(f"[Text Embedding] Extracting embeddings for: {text_list}")
        for text in text_list:
            print(f"[Text Embedding] Embedding column: {text}")
            tmp = self.get_cls_embedding_batch(self.train[text].tolist())
            text_set.append(tmp)
        print(f"[Text Embedding] All embeddings extracted. Shape: {[arr.shape for arr in text_set]}")
        return np.hstack(text_set)

    def df_load_train_text_emb(self, X_train, X_val, train_emb_list):
        if train_emb_list is None:
            train_emb_list = self.train_emb_list
        print(f"[DF Load] Loading train/val text embeddings for: {train_emb_list}")
        train_matrix_set = []
        val_matrix_set = []
        for col in train_emb_list:
            print(f"[DF Load] Processing train column: {col}")
            tmp = np.vstack(X_train[col].tolist())
            train_matrix_set.append(tmp)
        for col in train_emb_list:
            print(f"[DF Load] Processing val column: {col}")
            tmp = np.vstack(X_val[col].tolist())
            val_matrix_set.append(tmp)
        train_text_matrix = np.hstack(train_matrix_set)
        val_text_matrix = np.hstack(val_matrix_set)
        print(f"[DF Load] train_text_matrix shape: {train_text_matrix.shape}, val_text_matrix shape: {val_text_matrix.shape}")
        return train_text_matrix, val_text_matrix
    
    def df_load_test_text_emb(self, X_test, test_emb_list):
        if test_emb_list is None:
            test_emb_list = self.test_emb_list
        print(f"[DF Load Test] Loading test text embeddings for: {test_emb_list}")
        test_matrix_set = []
        for col in test_emb_list:
            print(f"[DF Load Test] Processing test column: {col}")
            tmp = np.vstack(X_test[col].tolist())
            test_matrix_set.append(tmp)
        test_text_matrix = np.hstack(test_matrix_set)
        print(f"[DF Load Test] test_text_matrix shape: {test_text_matrix.shape}")
        return test_text_matrix
    
    def scaled_matrix(self, matrix):
        scaler = StandardScaler()
        matrix_scaled = scaler.fit_transform(matrix)
        return matrix_scaled

    def concat_train_feature(self, X_train, X_val, train_text_matrix, 
                                                    val_text_matrix, 
                                                    feature_list):
        print(f"[Concat] Concatenating features: {feature_list}")
        train_feature_matrix = X_train[feature_list].to_numpy()
        val_feature_matrix = X_val[feature_list].to_numpy()

        # train_feature_matrix_scaled = self.scaled_matrix(train_feature_matrix)
        # val_feature_matrix_scaled = self.val_matrix(train_feature_matrix)

        train_full_matrix = np.hstack([train_text_matrix, train_feature_matrix])
        val_full_matrix = np.hstack([val_text_matrix, val_feature_matrix])
        print(f"[Concat] train_full_matrix shape: {train_full_matrix.shape}, val_full_matrix shape: {val_full_matrix.shape}")
        return train_full_matrix, val_full_matrix

    def concat_test_feature(self, test_text_matrix, feature_list):
        print(f"[Concat Test] Concatenating test features: {feature_list}")
        test_feature_matrix = self.test[feature_list].to_numpy()
        # test_feature_matrix_scaled = self.scaled_matrix(test_feature_matrix)
        test_full_matrix = np.hstack([test_text_matrix, test_feature_matrix])
        print(f"[Concat Test] test_full_matrix shape: {test_full_matrix.shape}")
        return test_full_matrix

In [ ]:
class FeatureEngineering:
    def __init__(self, train_matrix, val_matrix, test_matrix):
        self.train_matrix = train_matrix
        self.val_matrix = val_matrix
        self.test_matrix = test_matrix
        self.feature_df = None
    
    def scaled_matrix(self, matrix):
        scaler = StandardScaler()
        matrix_scaled = scaler.fit_transform(matrix)
        return matrix_scaled

    # TODO
    """
    def feature enginnering() -> Dataframe:
        특징 추출..
        
    def make_feature_df() -> Dataframe:
        ..
        return feature_df 

    etc..
    """


    def concat_feature(self, matrix, feature_df):
        if feature_df is None:
            feature_df = self.feature_df
        col_list=list(feature_df.columns)
        feature_matrix=feature_df[col_list].to_numpy()
        # feature_matrix_scared = self.scaled_matrix(feature_matrix)
        return np.hstack([matrix, feature_matrix])
        

In [3]:
dataset = Dataset("../data/train_paraemb_change_ngram.pkl", "../data/test_paraemb_change_ngram.pkl")
train, test = dataset.load_train_pickle()

[Init] Dataset initialized with train: ../data/train_paraemb_change_ngram.pkl, test: ../data/test_paraemb_change_ngram.pkl
[Load] Loading train pickle: ../data/train_paraemb_change_ngram.pkl
[Load] Loading test pickle: ../data/test_paraemb_change_ngram.pkl
[Load] train_col_list: ['paragraph_index', 'generated', 'paragraph_text_emb', 'adj_emb_change', 'title_emb', 'ngram_total', 'ngram_unique', 'ngram_max_freq', 'ngram_diversity', 'similarity']
[Load] test_col_list: ['ID', 'paragraph_index', 'paragraph_text_emb', 'title_emb', 'adj_emb_change', 'ngram_total', 'ngram_unique', 'ngram_max_freq', 'ngram_diversity', 'similarity']


In [4]:
text_list=[]
feature_list=["paragraph_index","adj_emb_change","ngram_total","ngram_unique","ngram_max_freq","ngram_diversity","similarity"]
train_emb_list=["paragraph_text_emb","title_emb"]
test_emb_list=["paragraph_text_emb", "title_emb"]
dataset.set_list(train_emb_list, test_emb_list, feature_list, text_list)

[Set] train_emb_list: ['paragraph_text_emb', 'title_emb'], 
 test_emb_list: ['paragraph_text_emb', 'title_emb'], 
feature_list: ['paragraph_index', 'adj_emb_change', 'ngram_total', 'ngram_unique', 'ngram_max_freq', 'ngram_diversity', 'similarity'], 
text_list: []


In [5]:
X_train, X_val, y_train, y_val = dataset.split_data(train)

[Split] Splitting data with stratify on 'generated'
[Split] X_train: (981091, 9), X_val: (245273, 9)


In [6]:
train_text_matrix, val_text_matrix = dataset.df_load_train_text_emb(X_train, X_val, train_emb_list)

[DF Load] Loading train/val text embeddings for: ['paragraph_text_emb', 'title_emb']
[DF Load] Processing train column: paragraph_text_emb
[DF Load] Processing train column: title_emb
[DF Load] Processing val column: paragraph_text_emb
[DF Load] Processing val column: title_emb
[DF Load] train_text_matrix shape: (981091, 1536), val_text_matrix shape: (245273, 1536)


In [7]:
train_full_matrix, val_full_matrix= dataset.concat_train_feature(X_train, X_val,
                                                                 train_text_matrix, 
                                                                 val_text_matrix, 
                                                                 feature_list)

[Concat] Concatenating features: ['paragraph_index', 'adj_emb_change', 'ngram_total', 'ngram_unique', 'ngram_max_freq', 'ngram_diversity', 'similarity']
[Concat] train_full_matrix shape: (981091, 1543), val_full_matrix shape: (245273, 1543)


## Train

In [8]:
# test.rename(columns={"paragraph_text_emb":"paragraph_text_emb"}, inplace=True)

In [8]:
test_text_matrix = dataset.df_load_test_text_emb(test, test_emb_list)
test_full_matrix = dataset.concat_test_feature(test_text_matrix, feature_list)

[DF Load Test] Loading test text embeddings for: ['paragraph_text_emb', 'title_emb']
[DF Load Test] Processing test column: paragraph_text_emb
[DF Load Test] Processing test column: title_emb
[DF Load Test] test_text_matrix shape: (1962, 1536)
[Concat Test] Concatenating test features: ['paragraph_index', 'adj_emb_change', 'ngram_total', 'ngram_unique', 'ngram_max_freq', 'ngram_diversity', 'similarity']
[Concat Test] test_full_matrix shape: (1962, 1543)


In [ ]:
# n_pos=np.sum(train["generated"]==1)
# n_neg=np.sum(train["generated"]==0)
# scale_pos_weight = n_neg / n_pos

In [9]:
xgb = XGBClassifier(random_state=42)
xgb.fit(train_full_matrix, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [10]:
val_probs = xgb.predict_proba(val_full_matrix)[:, 1]
auc = roc_auc_score(y_val, val_probs)
print(f"Validation AUC: {auc:.4f}")

Validation AUC: 0.9390


## Inference

In [11]:
probs = xgb.predict_proba(test_full_matrix)[:, 1]

## Submission

In [12]:
sample_submission = pd.read_csv('../data/sample_submission.csv', encoding='utf-8-sig')
sample_submission['generated'] = probs

sample_submission.to_csv(f'../output/baseline_submission_para_change_ngram_sim_scaled.csv', index=False)